# PkgEvalAnalysis

Latest pkgeval: https://s3.amazonaws.com/julialang-reports/nanosoldier/pkgeval/by_hash/9b7990d_vs_8611a64/report.html


In [55]:
using DataFrames, Feather

In [56]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    SyntaxError
    VisualRegression
    MissingDep
    BuildError
    UnsatReq
    BSONDataTypeField
    Belapsed
    StderrCheck
    EqChange
    
    # From PkgEval
    MissingDependency
    Inactivity
    MissingBinary
    Untestable
    LogLimit
    TimeLimit
    Syntax
    SegmentationFault
    
    # Specific ones
    Cassette
    LLVMVersion
    UninstalledDep
    ImageReference
    IOBufferWritable
    JSONUndefinedReference
    CcallType
    NoMethodZero
    CloseWrite
    ReturnType
    StdinBuffer
    BlockArraysIsLess
    TypeNameConstField
    CxxWrap
    InfinityArrays
    InfinityCardinal
    LibEnzyme
end


In [57]:
# Unpack the data unless it has already been unpacked

if !isdir("data")
    run(`tar -xvf data.tar.xz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

x data/
x data/primary.feather
x data/against.json
x data/against.feather
x data/primary.json


In [58]:
# Join the primary and against data

package_results = leftjoin(primary, against,
     on=:uuid, makeunique=true, indicator=:source);

In [59]:
# Filter out packages that started failing but didn't start failing when the RNG stream changed

fails = filter(test->test.source == "both" &&
                 test.status != test.status_1 &&
                 test.status in (":fail", ":kill"), package_results)

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

missing_binary_idx     = fails.reason .== ":binary_dependency"
inactivity_idx         = fails.reason .== ":inactivity"
missing_dependency_idx = fails.reason .== ":missing_dependency"
untestable_idx         = fails.reason .== ":untestable"
log_limit_idx          = fails.reason .== ":log_limit"
time_limit_idx         = fails.reason .== ":time_limit"
segfault_idx           = fails.reason .== ":segfault"

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

fails.why[missing_binary_idx]     .= MissingBinary
fails.why[inactivity_idx]         .= Inactivity
fails.why[missing_dependency_idx] .= MissingDependency
fails.why[untestable_idx]         .= Untestable
fails.why[log_limit_idx]          .= LogLimit
fails.why[time_limit_idx]         .= TimeLimit
fails.why[segfault_idx]           .= SegmentationFault


nothing

In [60]:
# Some utility functions

query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [61]:
query(fails, " Segmentation fault").name

1-element Vector{String}:
 "HierarchicalUtils"

In [62]:
deps = query(primary, "deprecated")
deps = query(deps, "Vararg")

,julia,compiled,name,uuid
,String,Bool,String,String
1,"v""1.8.0-beta2-412df00abf""",0,NMRTools,"UUID(""c2839fe3-f1de-42fc-897f-c1b0e8db4db6"")"
2,"v""1.8.0-beta2-412df00abf""",0,ROCKernels,"UUID(""7eb9e9f0-4bd3-4c4c-8bef-26bd9629d9b9"")"
3,"v""1.8.0-beta2-412df00abf""",0,DiscreteEvents,"UUID(""127e53a7-d08a-4bd9-afb0-daf0d2b65a85"")"
4,"v""1.8.0-beta2-412df00abf""",0,GeoEfficiency,"UUID(""b1f22607-7830-5ceb-9a8e-05ab1ac77008"")"
5,"v""1.8.0-beta2-412df00abf""",0,MEstimation,"UUID(""38feb9b4-a243-49d8-ab24-fe2508a85779"")"
6,"v""1.8.0-beta2-412df00abf""",0,StrRegex,"UUID(""2267b70d-4c4a-5da2-8395-98751c0d506a"")"
7,"v""1.8.0-beta2-412df00abf""",0,ImageCore,"UUID(""a09fc81d-aa75-5fe9-8630-4744c3626534"")"
8,"v""1.8.0-beta2-412df00abf""",0,SOM,"UUID(""172a5912-0be3-5bce-b68a-ea4f60b69b05"")"
9,"v""1.8.0-beta2-412df00abf""",0,SpatialBoundaries,"UUID(""8d2ba62a-3d23-4a2b-b692-6b63e9267be2"")"


In [63]:
# Here we pattern match certain test errors and categorize them based on that.

# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "Error: doctest failure in ", DocTest)

update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, r"Expression: (\S*) ≈ (\S*)", ApproxError) 
update_reason!(fails, "Expression: ≈(", ApproxError)
    
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, " ambiguities found", NewAmbiguity)

update_reason!(fails, "Unsatisfiable requirements detected for package", UnsatReq)





update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)
update_reason!(fails, "LoadError: syntax", SyntaxError)

update_reason!(fails, "Image did not match reference image", VisualRegression)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

update_reason!(fails, "Error building ", BuildError)
update_reason!(fails, "isempty(stderr_content)", StderrCheck)

update_reason!(fails, r"Expression: (\N.*?) == ", EqChange)
update_reason!(fails, "signal (11): Segmentation fault", SegmentationFault)





# Specific ones for this release
update_reason!(fails, "Cassette.overdub", Cassette)
update_reason!(fails, "libLLVM-12jl.so", LLVMVersion)
update_reason!(fails, "You may have a partially installed environment", UninstalledDep)
update_reason!(fails, "- REFERENCE -------------------", ImageReference)
update_reason!(fails, "ArgumentError: ensureroom failed, IOBuffer is not writeable", IOBufferWritable)
update_reason!(fails, "JSON.Writer ~/.julia/packages/JSON/NeJ9k/src/Writer.jl:285", JSONUndefinedReference)
update_reason!(fails, "ccall method definition: argument 1 type doesn't correspond to a C type", CcallType)
update_reason!(fails, "no method matching zero", NoMethodZero)
update_reason!(fails, "closewrite not defined", CloseWrite)
update_reason!(fails, "no method matching return_type", ReturnType)
update_reason!(fails, "type IOStream has no field buffer", StdinBuffer)
update_reason!(fails, "UndefVarError: _isless not defined", BlockArraysIsLess)
update_reason!(fails, "setfield!: const field .names of type TypeName cannot be changed", TypeNameConstField)
update_reason!(fails, "C++ exception while wrapping module", CxxWrap)
update_reason!(fails, "no method matching Infinities.Infinity(::Float64)", InfinityArrays)
update_reason!(fails, "InfiniteArrays.Infinity(::Int64)", InfinityArrays)
update_reason!(fails, "no method matching Infinities.InfiniteCardinal", InfinityCardinal)
update_reason!(fails, "libEnzyme-12.so", LibEnzyme)
update_reason!(fails, " is non-positive", InfinityArrays)







nothing

In [74]:
# Packages that have gotten  

issues_opened = [
    "StaticKernels" # https://github.com/stev47/StaticKernels.jl/issues/5
    "Polyhedra" # https://github.com/JuliaPolyhedra/Polyhedra.jl
    "TexasHoldEm" # https://github.com/charleskawczynski/TexasHoldem.jl/issues/106
    "RedefStructs" # https://github.com/FedericoStra/RedefStructs.jl/issues/4
    "Memento" # https://github.com/invenia/Memento.jl/issues/184
    "GarishPrint" # https://github.com/JuliaLang/Pkg.jl/issues/3015
    "SpecialFunctions" # https://github.com/JuliaLang/julia/issues/44452
    "RangeHelpers" # https://github.com/jw3126/RangeHelpers.jl/issues/2
    "SIMD" # https://github.com/eschnett/SIMD.jl/pull/96
    "HierarchicalUtils" # https://github.com/JuliaLang/julia/issues/44501
    "HarmonicOrthogonalPolynomials" # https://github.com/JuliaArrays/InfiniteArrays.jl/issues/91
    "InfiniteArrays" # https://github.com/JuliaArrays/InfiniteArrays.jl/issues/89
    "TensorKitManifolds" # https://github.com/JuliaLang/julia/issues/44712
    "Enzyme" # https://github.com/EnzymeAD/Enzyme.jl/issues/250
    "BEASTXMLConstructor" # https://github.com/Julia-BEAST-utils/BEASTXMLConstructor.jl/issues/5
    "Dash" # https://github.com/JuliaLang/julia/issues/44732
    "EquationsOfStateOfSolids" # https://github.com/MineralsCloud/EquationsOfStateOfSolids.jl/issues/146
 ]

likely_tol = [
    "TaylorModels"
];

fixed = [
    "GeoDataFrames" # https://github.com/evetion/GeoDataFrames.jl/pull/30
    "Cassette" # https://github.com/JuliaLabs/Cassette.jl/commit/8219e43264543da43e4d71bd860d09fc6a016630
    "LiterateTest" # https://github.com/tkf/LiterateTest.jl/commit/1ca4e69abedfcfb609586d438835479f03ad831f
    "ProfileSVG" # https://github.com/kimikage/ProfileSVG.jl/pull/64
    "TORA" # https://github.com/JuliaRobotics/TORA.jl/commit/c33f3ef5f0d5fb8cec3cf1ee7ffb2f931d761af2
]




ignored_packages = [
]

Any[]

In [75]:
# Total package failures that we haven't categorized

total_unknown(fails)

23

# Examples

Here are some examples of how one might do queries and categorize errors

In [76]:
# Unknown failures

filter(x -> x.why == Unknown && 
       !(x.name in issues_opened) && 
       !(x.name in likely_tol) && 
       !(x.name in ignored_packages) && 
       !(x.name in fixed), 
    fails)

,julia,compiled,name,uuid
,String,Bool,String,String
1,"v""1.8.0-beta2-412df00abf""",0,PkgDeps,"UUID(""839e9fc8-855b-5b3c-a3b7-2833d3dd1f59"")"
2,"v""1.8.0-beta2-412df00abf""",0,NMRTools,"UUID(""c2839fe3-f1de-42fc-897f-c1b0e8db4db6"")"
3,"v""1.8.0-beta2-412df00abf""",0,CxxWrap,"UUID(""1f15a43c-97ca-5a2a-ae31-89f07a497df4"")"
4,"v""1.8.0-beta2-412df00abf""",0,SnoopCompile,"UUID(""aa65fe97-06da-5843-b5b1-d5d13cad87d2"")"
5,"v""1.8.0-beta2-412df00abf""",0,LRSLib,"UUID(""262c1cb6-76e2-5873-868b-19ece3183cc5"")"
6,"v""1.8.0-beta2-412df00abf""",0,StatProfilerHTML,"UUID(""a8a75453-ed82-57c9-9e16-4cd1196ecbf5"")"
7,"v""1.8.0-beta2-412df00abf""",0,FlxQTL,"UUID(""ede81e08-c6d8-4fe3-94c2-f928d9d678ed"")"
8,"v""1.8.0-beta2-412df00abf""",0,TraitSimulation,"UUID(""dec3038e-29bc-11e9-2207-9f3d5855a202"")"
9,"v""1.8.0-beta2-412df00abf""",0,MIRTjim,"UUID(""170b2178-6dee-4cb0-8729-b3e8b57834cc"")"


In [67]:
# Categorize

z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    n > 0 && push!(z, (n, i))
end
sort!(z; rev=true)

27-element Vector{Any}:
 (24, EqChange)
 (23, Unknown)
 (17, NewExceptionType)
 (16, CcallType)
 (13, UninstalledDep)
 (9, NewAmbiguity)
 (7, LibEnzyme)
 (6, CxxWrap)
 (6, ApproxError)
 (5, InfinityCardinal)
 (4, ReturnType)
 (3, InfinityArrays)
 (3, DocTest)
 ⋮
 (2, NoMethodZero)
 (2, TimeLimit)
 (2, MissingBinary)
 (2, UnsatReq)
 (1, TypeNameConstField)
 (1, JSONUndefinedReference)
 (1, IOBufferWritable)
 (1, ImageReference)
 (1, SegmentationFault)
 (1, Inactivity)
 (1, TestAmbiguities)
 (1, InferredFailure)

In [70]:
filter(row -> row.why == MissingBinary, fails)

,julia,compiled,name,uuid
,String,Bool,String,String
1,"v""1.8.0-beta2-412df00abf""",0,ClimateTools,"UUID(""4f4ee721-4970-5af2-8560-6c1d960e3231"")"
2,"v""1.8.0-beta2-412df00abf""",0,TurbulenceConvection,"UUID(""8e072fc4-01f8-44fb-b9dc-f9336c367e6b"")"


In [69]:
OrbitalTrajectories

LoadError: UndefVarError: OrbitalTrajectories not defined